In [8]:
# Импорт библиотек для работы с данными
import pandas as pd
import numpy as np
import scipy.stats as stats

# Импорт библиотек для визуализации данных
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Импорт библиотек для пред обработки данных
from sklearn.preprocessing import MaxAbsScaler, RobustScaler

# Импорт библиотек для отбора признаков
from sklearn.feature_selection import RFE, RFECV, SelectFromModel, SelectFpr, chi2

# Импорт библиотек линейной регрессии
import statsmodels.api as sm
from statsmodels.regression.recursive_ls import RecursiveLS
from statsmodels.regression.linear_model import OLS, GLS, GLSAR, WLS


# Импорт библиотек для машинного обучения
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [9]:
data = pd.read_csv("input/data.csv", delimiter=";")
# data = data.drop(columns=['Год'])
data.head()

,Год,y1,y2,y3,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11
0,1998,87211.83,328497.9,638450.60,228548.4,803.2,5780.87,1212184.0,33.8,83359.41,25719.20,6171.0,6642.0,111458.0,1393.30
1,1999,119302.33,695059.8,789466.92,488395.1,787.6,5823.01,1272744.0,35.2,87820.24,25262.15,7260.0,7814.0,185861.0,1807.80
2,2000,156215.00,1159034.0,962057.00,748241.8,1472.8,5865.15,1444737.0,32.4,92700.80,25720.20,8067.0,9194.0,309534.0,2185.00
3,2001,173839.00,1370182.8,1393532.20,1008088.5,1154.6,5907.28,1841258.0,30.4,98088.36,24905.88,5545.0,12637.0,418289.0,2385.26
4,2002,220396.00,1767476.7,1771073.00,1267935.2,1508.7,5961.24,2255912.0,35.1,104100.59,25084.01,6932.0,13817.0,589139.0,2918.55


In [10]:
# Преобразуем столбец с датой в формат datetime
data['Год'] = pd.to_datetime(data['Год'].astype(str), format='%Y')
data.head()

,Год,y1,y2,y3,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11
0,1998-01-01,87211.83,328497.9,638450.60,228548.4,803.2,5780.87,1212184.0,33.8,83359.41,25719.20,6171.0,6642.0,111458.0,1393.30
1,1999-01-01,119302.33,695059.8,789466.92,488395.1,787.6,5823.01,1272744.0,35.2,87820.24,25262.15,7260.0,7814.0,185861.0,1807.80
2,2000-01-01,156215.00,1159034.0,962057.00,748241.8,1472.8,5865.15,1444737.0,32.4,92700.80,25720.20,8067.0,9194.0,309534.0,2185.00
3,2001-01-01,173839.00,1370182.8,1393532.20,1008088.5,1154.6,5907.28,1841258.0,30.4,98088.36,24905.88,5545.0,12637.0,418289.0,2385.26
4,2002-01-01,220396.00,1767476.7,1771073.00,1267935.2,1508.7,5961.24,2255912.0,35.1,104100.59,25084.01,6932.0,13817.0,589139.0,2918.55


In [11]:
# Устанавливаем столбец с датой в качестве индекса DataFrame
data.set_index('Год', inplace=True)
data.head()

,y1,y2,y3,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11
Год,,,,,,,,,,,,,,
1998-01-01,87211.83,328497.9,638450.60,228548.4,803.2,5780.87,1212184.0,33.8,83359.41,25719.20,6171.0,6642.0,111458.0,1393.30
1999-01-01,119302.33,695059.8,789466.92,488395.1,787.6,5823.01,1272744.0,35.2,87820.24,25262.15,7260.0,7814.0,185861.0,1807.80
2000-01-01,156215.00,1159034.0,962057.00,748241.8,1472.8,5865.15,1444737.0,32.4,92700.80,25720.20,8067.0,9194.0,309534.0,2185.00
2001-01-01,173839.00,1370182.8,1393532.20,1008088.5,1154.6,5907.28,1841258.0,30.4,98088.36,24905.88,5545.0,12637.0,418289.0,2385.26
2002-01-01,220396.00,1767476.7,1771073.00,1267935.2,1508.7,5961.24,2255912.0,35.1,104100.59,25084.01,6932.0,13817.0,589139.0,2918.55


In [12]:
# Используем метод resample() для ресэмплинга данных по кварталам
data_quarterly = data.resample('Q').mean()
data_quarterly.head()

,y1,y2,y3,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11
Год,,,,,,,,,,,,,,
1998-03-31,87211.83,328497.9,638450.60,228548.4,803.2,5780.87,1212184.0,33.8,83359.41,25719.20,6171.0,6642.0,111458.0,1393.3
1998-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-03-31,119302.33,695059.8,789466.92,488395.1,787.6,5823.01,1272744.0,35.2,87820.24,25262.15,7260.0,7814.0,185861.0,1807.8


In [13]:
# Используем метод resample() с параметром fill_method для достройки данных
data_quarterly_all_years = data_quarterly.resample('Q').mean()
data_quarterly_all_years.head()

,y1,y2,y3,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11
Год,,,,,,,,,,,,,,
1998-03-31,87211.83,328497.9,638450.60,228548.4,803.2,5780.87,1212184.0,33.8,83359.41,25719.20,6171.0,6642.0,111458.0,1393.3
1998-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-03-31,119302.33,695059.8,789466.92,488395.1,787.6,5823.01,1272744.0,35.2,87820.24,25262.15,7260.0,7814.0,185861.0,1807.8


In [14]:
# Используем метод resample() для ресэмплинга данных по кварталам
data_quarterly = data.resample('Q').ffill()

# Используем метод resample() с параметром fill_method для достройки данных
data_quarterly_all_years = data_quarterly.resample('Q').ffill()

# Сбрасываем индекс и выводим результаты
data_quarterly_all_years = data_quarterly_all_years.reset_index()
print(data_quarterly_all_years.head())

         Год         y1        y2         y3        x1     x2       x3   
0 1998-03-31   87211.83  328497.9  638450.60  228548.4  803.2  5780.87  \
1 1998-06-30   87211.83  328497.9  638450.60  228548.4  803.2  5780.87   
2 1998-09-30   87211.83  328497.9  638450.60  228548.4  803.2  5780.87   
3 1998-12-31   87211.83  328497.9  638450.60  228548.4  803.2  5780.87   
4 1999-03-31  119302.33  695059.8  789466.92  488395.1  787.6  5823.01   

          x4    x5        x6        x7      x8      x9       x10     x11  
0  1212184.0  33.8  83359.41  25719.20  6171.0  6642.0  111458.0  1393.3  
1  1212184.0  33.8  83359.41  25719.20  6171.0  6642.0  111458.0  1393.3  
2  1212184.0  33.8  83359.41  25719.20  6171.0  6642.0  111458.0  1393.3  
3  1212184.0  33.8  83359.41  25719.20  6171.0  6642.0  111458.0  1393.3  
4  1272744.0  35.2  87820.24  25262.15  7260.0  7814.0  185861.0  1807.8  
